# 🐍 Day 1-1: Regression & Pipeline 기초

머신러닝 프로젝트는 단순히 모델을 훈련하는 것 이상의 과정을 포함합니다. 데이터를 불러와 정제하고, 특성을 추출하며, 모델을 학습시키고 평가하는 등 여러 단계를 거치게 됩니다.

이번 세션에서는 이 모든 과정을 체계적으로 관리하는 **ML 워크플로**의 중요성을 이해하고, `scikit-learn`의 강력한 도구인 **`Pipeline`**을 사용하여 재현성 높고 효율적인 코드를 작성하는 방법을 배웁니다.

나아가 데이터의 관계를 학습하여 연속적인 값을 예측하는 **회귀(Regression)** 모델의 가장 기본이 되는 선형, 다중, 다항 회귀를 깊이 있게 다루며 모델링의 첫걸음을 떼게 됩니다.


---
### 🚀 1. ML 워크플로와 `Pipeline`의 중요성
효과적인 머신러닝 시스템을 구축하기 위해서는 일관되고 재현 가능한 작업 흐름, 즉 **워크플로(Workflow)**를 만드는 것이 매우 중요합니다.

#### 1.1 개념: 왜 체계적인 워크플로가 필요한가? 🤔

상상해 보세요. 여러 데이터 파일을 다루고, 결측치를 채우고, 텍스트 데이터를 숫자로 바꾸고, 스케일링을 적용하는 등 수많은 전처리 단계를 거친다고 가정해 봅시다. 이 과정이 뒤죽박죽이라면 어떨까요?

-   **실수 유발**: 어떤 데이터에 어떤 전처리를 적용했는지 잊어버리기 쉽습니다.
-   **데이터 누수 (Data Leakage)**: 훈련 데이터의 정보가 검증 데이터나 테스트 데이터에 흘러 들어가 모델의 성능을 비현실적으로 좋게 평가하는 치명적인 실수를 유발할 수 있습니다. 예를 들어, 전체 데이터의 평균값으로 결측치를 채운 뒤 훈련/테스트 데이터로 나누면, 테스트 데이터에 훈련 데이터의 정보(평균값)가 이미 포함된 셈입니다.
-   **재현성 저하** : 동일한 결과를 다시 만들어내기 어렵습니다.

`scikit-learn`의 **`Pipeline`** 은 이러한 문제들을 해결하기 위해 데이터 전처리 단계와 모델 학습 단계를 하나의 연속된 작업으로 묶어주는 훌륭한 도구입니다. 파이프라인을 사용하면 전체 워크플로를 하나의 객체처럼 다룰 수 있어 코드가 간결해지고, 데이터 누수를 방지하며, 재현성을 크게 향상시킬 수 있습니다.

특히, 각기 다른 전처리 방법이 필요한 특성(피처)들을 한 번에 처리하기 위해 **`ColumnTransformer`** 를 `Pipeline`과 함께 사용하는 것이 일반적입니다.


#### 1.2 코드 예제: `Pipeline`과 `ColumnTransformer`로 워크플로 구축하기

**Ames Housing** 데이터셋을 사용하여, 수치형 특성과 범주형 특성을 각각 다르게 전처리하고 선형 회귀 모델을 학습시키는 파이프라인을 만들어 보겠습니다.

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [65]:
from sklearn.datasets import fetch_openml
housing = fetch_openml(name="house_prices", as_frame=True)
df = housing.data.join(housing.target)

In [66]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [74]:
# 독립 변수(X)와 종속 변수(y) 분리
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# 훈련/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
# 2. 전처리 파이프라인 정의
# 수치형 데이터 전처리: 결측치를 평균으로 채우고, 표준화 스케일링 적용
numeric_features = ['LotArea', 'OverallQual']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])


In [76]:
# 범주형 데이터 전처리: 결측치를 'missing'으로 채우고, 원-핫 인코딩 적용
categorical_features = ['BldgType', 'Neighborhood']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [77]:
# 3. ColumnTransformer로 전처리 파이프라인 통합
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

In [78]:
# 4. 최종 파이프라인 구축 (전처리기 + 모델)
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

In [79]:
# 5. 모델 학습 및 예측
model_pipeline.fit(X_train, y_train)
y_pred = model_pipeline.predict(X_test)

In [80]:
# 6. 평가
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"파이프라인을 이용한 모델의 RMSE: ${rmse:,.2f}")


파이프라인을 이용한 모델의 RMSE: $42,440.26


In [11]:
# 파이프라인 내부 확인
print("Fitted Pipeline Regressor Coef:")
print(model_pipeline.named_steps['regressor'].coef_)

Fitted Pipeline Regressor Coef:
[21661.80354372 29661.81157082  3813.34277592 -7322.35579133
  9705.32493763 -6196.31192222  9712.53745725 -7322.35579133
 -6196.31192222  9705.32493763 -5899.19468134]


#### 1.3 연습 문제

1.  위 예제에서 `SimpleImputer`의 `strategy`를 수치형 데이터는 `'median'`으로, 범주형 데이터는 `'most_frequent'`로 변경하여 파이프라인을 다시 구성하고 학습시켜 보세요.


2.  `LinearRegression` 모델 대신 `Ridge` 회귀 모델(`from sklearn.linear_model import Ridge`)을 사용하는 새로운 파이프라인 `ridge_pipeline`을 만들어 보세요.

-----

### 💎 2. 선형 회귀 (Linear Regression)

선형 회귀는 가장 기본적이면서도 강력한 회귀 알고리즘입니다. 변수들 사이의 '선형적인' 관계를 모델링하여 예측을 수행합니다.



#### 2.1 개념: 기본 원리와 가정 📐

**단순 선형 회귀 (Simple Linear Regression)** 는 하나의 독립 변수 $X$가 종속 변수 $Y$에 미치는 영향을 직선 형태로 모델링합니다.

$$ Y = \beta_0 + \beta_1X + \epsilon $$

  - $Y$: 예측하려는 값 (종속 변수, 예: 주택 가격)
  - $X$: 예측에 사용할 값 (독립 변수, 예: 주택의 면적)
  - $\\beta\_0$: **절편 (Intercept)**. $X$가 0일 때의 $Y$ 값.
  - $\\beta\_1$: **계수 (Coefficient)** 또는 **기울기 (Slope)**. $X$가 1단위 증가할 때 $Y$의 변화량.
  - $\\epsilon$: **오차항 (Error Term)**. 모델이 설명하지 못하는 무작위 오차.

선형 회귀의 목표는 실제 값과 모델의 예측 값 사이의 오차(잔차) 제곱의 합(Sum of Squared Errors)을 최소화하는 $\\beta\_0$와 $\\beta\_1$을 찾는 것입니다. 이를 **최소제곱법(Ordinary Least Squares, OLS)** 이라고 합니다.


#### 2.2 코드 예제: 단일 변수 선형 회귀와 시각화

주택의 지상층 생활 면적(`GrLivArea`)만으로 주택 가격(`SalePrice`)을 예측하는 단순 선형 회귀 모델을 만들어 보겠습니다. 데이터는 캐글의 Ames Housing 데이터셋을 사용합니다.

In [82]:
import pandas as pd
import plotly.express as px
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [83]:
# 독립/종속 변수 선택
X = df[['GrLivArea']] # 2D 형태로 입력
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [84]:
# 모델 생성 및 학습
lr_simple = LinearRegression()
lr_simple.fit(X_train, y_train)

# 계수 및 절편 확인
print(f"계수 (기울기): {lr_simple.coef_[0]:.4f}")
print(f"절편: {lr_simple.intercept_:.4f}")

계수 (기울기): 100.7528
절편: 27237.2792


In [85]:
# 예측 및 평가
y_pred = lr_simple.predict(X_test)
print(f"R-squared (결정계수): {r2_score(y_test, y_pred):.4f}")

R-squared (결정계수): 0.5508


In [86]:
# 시각화 (Plotly Express 사용)
# 원본 데이터와 회귀선을 함께 그리기
fig = px.scatter(x=X_train['GrLivArea'], y=y_train,
                 labels={'x': '방 개수 (GrLivArea)', 'y': '주택 가격 (SalePrice)'},
                 title='단순 선형 회귀: 방 개수와 주택 가격',
                 template='plotly_white')

# 회귀선 추가 (Plotly의 trendline 기능 활용)
fig.update_traces(marker=dict(size=8, opacity=0.7))
fig.add_trace(px.line(x=X_test['GrLivArea'], y=y_pred).data[0]) # 예측된 라인
fig.data[1].line.color = 'red'
fig.data[1].name = '회귀선'

fig.show()

#### 2.3 연습 문제

1.  위 예제에서 `GrLivArea` 대신 저소득층 비율을 나타내는 `LSTAT` 변수를 사용하여 단순 선형 회귀 모델을 학습시키고, R-squared 값을 비교해 보세요.

2.  `LSTAT` 모델의 계수(coefficient)는 양수일까요, 음수일까요? 그 이유는 무엇일지 해석해 보세요.



-----

### 🧩 3. 다중 선형 회귀 (Multiple Linear Regression)

현실의 문제는 여러 요인이 복합적으로 작용합니다. 다중 선형 회귀는 두 개 이상의 독립 변수를 사용하여 종속 변수를 예측하는 모델입니다.

#### 3.1 개념: 여러 개의 특성 활용하기 📈

모델의 수식은 변수의 개수만큼 확장됩니다.

$$ Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + ... + \beta_pX_p + \epsilon $$

여기서 각 계수 $\\beta\_i$는 **다른 모든 변수들이 일정하다고 가정할 때**, 해당 변수 $X\_i$가 1단위 증가할 때 $Y$에 미치는 영향을 나타냅니다. 이 "다른 변수는 일정하다"는 가정이 해석에 매우 중요합니다.

다중 회귀는 더 많은 정보를 사용하므로 단순 회귀보다 높은 예측 성능을 보이는 경우가 많습니다. 하지만 불필요한 변수를 추가하면 오히려 성능이 저하될 수 있으며, 변수 간 관계(다중공선성)도 고려해야 합니다.

#### 3.2 코드 예제: 다중 선형 회귀 모델링

이번에는 방 개수(`GrLivArea`), 범죄율(`CRIM`), 학생-교사 비율(`PTRATIO`) 세 가지 변수를 모두 사용하여 주택 가격을 예측해 보겠습니다.

In [88]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# 데이터는 위에서 사용한 df를 그대로 사용합니다.
# 독립 변수 선택 - Ames Housing 데이터셋에서 판매가격에 영향을 줄만한 변수들
features = ['GrLivArea', 'OverallQual', 'YearBuilt']
X_multi = df[features]
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X_multi, y, test_size=0.3, random_state=42)

In [89]:
# 다중 회귀 모델 생성 및 학습
lr_multi = LinearRegression()
lr_multi.fit(X_train, y_train)

LinearRegression()

In [90]:
# 계수 확인
print("다중 회귀 모델 계수:")
for feature, coef in zip(features, lr_multi.coef_):
    print(f"  - {feature}: {coef:.4f}")
print(f"절편: {lr_multi.intercept_:.4f}")

다중 회귀 모델 계수:
  - GrLivArea: 58.5720
  - OverallQual: 24755.1006
  - YearBuilt: 515.0353
절편: -1075093.8347


In [91]:
# 예측 및 평가
y_pred_multi = lr_multi.predict(X_test)
print(f"다중 회귀 R-squared: {r2_score(y_test, y_pred_multi):.4f}")

다중 회귀 R-squared: 0.7618


단순 회귀 모델의 R-squared 값과 비교해 보면, 여러 변수를 사용했을 때 설명력이 더 높아진 것을 확인할 수 있습니다.

#### 3.3 연습 문제

1.  위 다중 회귀 모델에 `NOX`(산화질소 농도) 변수를 추가하여 R-squared 값이 어떻게 변하는지 확인해 보세요.


2.  `GrLivArea`의 계수가 단순 회귀 때와 다중 회귀 때 달라진 것을 볼 수 있습니다. 그 이유는 무엇일지 생각해 보세요.


-----
### 🎢 4. 다항 회귀 (Polynomial Regression)와 과적합

데이터의 관계가 항상 직선은 아닙니다. 다항 회귀는 독립 변수를 거듭제곱하여 생성된 새로운 특성들을 추가함으로써 비선형 관계를 모델링하는 방법입니다.

#### 4.1 개념: 비선형 관계를 포착하는 방법 〰️

예를 들어 2차 다항 회귀는 기존 변수 $X$ 외에 $X^2$을 새로운 변수로 추가하여 선형 회귀를 수행합니다.

$$ Y = \beta_0 + \beta_1X + \beta_2X^2 + \epsilon $$

이는 여전히 계수( $\beta$ )에 대해 선형이기 때문에 선형 회귀의 틀 안에서 해결할 수 있습니다.

`scikit-learn`의 `PolynomialFeatures`를 사용하여 간단히 구현할 수 있습니다.

하지만 차수(degree)를 너무 높이면 모델이 훈련 데이터에만 과도하게 맞춰져 새로운 데이터에 대한 예측 성능이 떨어지는 **과적합(Overfitting)** 문제가 발생할 수 있습니다. 

모델이 너무 복잡해져 데이터의 실제 패턴이 아닌 노이즈까지 학습하기 때문입니다.

우리는 `OverallQual`(전반적인 품질) 변수와 `SalePrice` 간의 관계를 다항 회귀로 모델링하여 이러한 개념을 실습해보겠습니다.

#### 4.2 코드 예제: `PolynomialFeatures`를 이용한 다항 회귀

`OverallQual`(전반적인 품질)과 `SalePrice` 간의 비선형 관계를 다항 회귀로 모델링하고, 차수에 따른 과적합 현상을 시각적으로 확인해 보겠습니다.

In [93]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# OverallQual 변수 사용 (전반적인 품질 - 낮을수록 저품질)
X = df[['OverallQual']]
y = df['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [24]:
# RMSE 계산 함수 정의
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [95]:
# 시각화를 위한 데이터 정렬
X_range = np.linspace(X.min(), X.max(), 100).reshape(-1, 1)
X_range = pd.DataFrame(X_range, columns=['OverallQual'])

In [96]:
# Degree 1 모델
poly_reg_1 = make_pipeline(PolynomialFeatures(degree=1), LinearRegression())
poly_reg_1.fit(X_train, y_train)
y_pred_range_1 = poly_reg_1.predict(X_range)
y_pred_train_1 = poly_reg_1.predict(X_train)
y_pred_test_1 = poly_reg_1.predict(X_test)
train_rmse_1 = calculate_rmse(y_train, y_pred_train_1)
test_rmse_1 = calculate_rmse(y_test, y_pred_test_1)
print(f"Degree 1 - Train RMSE: {train_rmse_1:.4f}, Test RMSE: {test_rmse_1:.4f}")

Degree 1 - Train RMSE: 48087.6739, Test RMSE: 49834.0154


In [97]:
# Degree 2 모델
poly_reg_2 = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_reg_2.fit(X_train, y_train)
y_pred_range_2 = poly_reg_2.predict(X_range)
y_pred_train_2 = poly_reg_2.predict(X_train)
y_pred_test_2 = poly_reg_2.predict(X_test)
train_rmse_2 = calculate_rmse(y_train, y_pred_train_2)
test_rmse_2 = calculate_rmse(y_test, y_pred_test_2)
print(f"Degree 2 - Train RMSE: {train_rmse_2:.4f}, Test RMSE: {test_rmse_2:.4f}")

Degree 2 - Train RMSE: 44884.1997, Test RMSE: 45461.5718


In [113]:
# Degree 10 모델
poly_reg_10 = make_pipeline(PolynomialFeatures(degree=10), LinearRegression())
poly_reg_10.fit(X_train, y_train)
y_pred_range_10 = poly_reg_10.predict(X_range)
y_pred_train_10 = poly_reg_10.predict(X_train)
y_pred_test_10 = poly_reg_10.predict(X_test)
train_rmse_10 = calculate_rmse(y_train, y_pred_train_10)ㅋ
test_rmse_10 = calculate_rmse(y_test, y_pred_test_10)
print(f"Degree 10 - Train RMSE: {train_rmse_10:.4f}, Test RMSE: {test_rmse_10:.4f}")

SyntaxError: invalid syntax (2477402172.py, line 7)

In [114]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, 
                    subplot_titles=['Degree 1 (선형)', 'Degree 2 (2차)', 'Degree 5 (5차)'],
                    shared_yaxes=True)

# Degree 1 서브플롯
fig.add_trace(go.Scatter(x=X_train['OverallQual'], y=y_train, 
                         mode='markers', opacity=0.6, 
                         name='훈련 데이터', 
                         marker=dict(color='lightblue'),
                         showlegend=True), row=1, col=1)
fig.add_trace(go.Scatter(x=X_range['OverallQual'], y=y_pred_range_1,
                         mode='lines',
                         name=f'Degree 1 (Test RMSE: {test_rmse_1:.2f})',
                         line=dict(color='orange', width=3),
                         showlegend=True), row=1, col=1)

# Degree 2 서브플롯
fig.add_trace(go.Scatter(x=X_train['OverallQual'], y=y_train, 
                         mode='markers', opacity=0.6, 
                         name='훈련 데이터', 
                         marker=dict(color='lightblue'),
                         showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=X_range['OverallQual'], y=y_pred_range_2,
                         mode='lines',
                         name=f'Degree 2 (Test RMSE: {test_rmse_2:.2f})',
                         line=dict(color='red', width=3),
                         showlegend=True), row=1, col=2)

# Degree 10 서브플롯
fig.add_trace(go.Scatter(x=X_train['OverallQual'], y=y_train, 
                         mode='markers', opacity=0.6, 
                         name='훈련 데이터', 
                         marker=dict(color='lightblue'),
                         showlegend=False), row=1, col=3)
fig.add_trace(go.Scatter(x=X_range['OverallQual'], y=y_pred_range_10,
                         mode='lines',
                         name=f'Degree 10 (Test RMSE: {test_rmse_10:.2f})',
                         line=dict(color='green', width=3),
                         showlegend=True), row=1, col=3)

# 레이아웃 업데이트
fig.update_layout(title_text='차수에 따른 다항 회귀 모델 비교',
                  height=400)
fig.update_xaxes(title_text='전반적인 품질(OverallQual)')
fig.update_yaxes(title_text='주택 가격', col=1)

fig.show()

위 그래프를 보면,

  - **Degree 1 (주황색)**: 선형 모델로, 데이터의 곡선 패턴을 잘 잡아내지 못합니다(과소적합).
  - **Degree 2 (빨간색)**: 2차 곡선으로, 데이터의 전반적인 추세를 잘 설명하며 테스트 오차도 낮습니다.
  - **Degree 10 (초록색)**: 훈련 데이터 포인트를 거의 완벽하게 따라가지만, 조금 구불구불한 형태를 보입니다. 이는 훈련 데이터의 노이즈까지 학습한 결과이며, 이 예제에서는 새로운 데이터에 대한 일반화 성능(RMSE 최소화)이 조금 올라갔지만, 오히려 떨어지는 다른 데이터셋에서는 과적합 상태가 발생할 가능성도 있습니다.

#### 4.3 연습 문제

1.  `PolynomialFeatures`의 `degree`를 3으로 설정하여 모델을 만들고, Test RMSE를 Degree 2, 10 모델과 비교해 보세요.

2.  차수를 1부터 15까지 변화시키면서 각 차수별 Train RMSE와 Test RMSE를 계산하고, 이를 꺾은선 그래프로 그려보세요. 어느 지점에서 Test RMSE가 다시 증가하기 시작하는지(과적합이 시작되는 지점) 확인해 보세요.


-----

## 🔑 정답지

### 1.3 연습 문제 정답 보기

1.  **Imputer 전략 변경**

In [ ]:
# 수치형: median
numeric_transformer_median = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# 범주형: most_frequent
categorical_transformer_freq = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor_new = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_median, numeric_features),
        ('cat', categorical_transformer_freq, categorical_features)])

model_pipeline_new = Pipeline(steps=[('preprocessor', preprocessor_new),
                                      ('regressor', LinearRegression())])
model_pipeline_new.fit(X_train, y_train)
# 이후 예측 및 평가는 동일

2.  **Ridge 모델 파이프라인**

In [ ]:
from sklearn.linear_model import Ridge

ridge_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor), # 기존 preprocessor 재사용
    ('regressor', Ridge(alpha=1.0)) # Ridge 모델로 교체
])
ridge_pipeline.fit(X_train, y_train)


### 2.3 연습 문제 정답 보기

1.  **LSTAT 변수 사용**

In [ ]:
X_lstat = df[['LSTAT']]
y = df['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X_lstat, y, test_size=0.3, random_state=42)

lr_lstat = LinearRegression().fit(X_train, y_train)
y_pred_lstat = lr_lstat.predict(X_test)
print(f"LSTAT 모델 R-squared: {r2_score(y_test, y_pred_lstat):.4f}")
# GrLivArea(RM) 모델의 R-squared(약 0.44)보다 높은 값(약 0.52)이 나옵니다.
# 즉, LSTAT 변수가 주택 가격을 더 잘 설명하는 경향이 있습니다.

2.  **LSTAT 계수 해석**
    계수는 음수가 나옵니다 (`lr_lstat.coef_[0]` 확인 시 약 -0.9 \~ -1.0). 이는 저소득층 비율(`LSTAT`)이 높을수록 주택 가격(`SalePrice`)은 낮아지는 경향이 있음을 의미하며, 이는 상식적으로 타당한 해석입니다.

### 3.3 연습 문제 정답 보기

1.  **NOX 변수 추가**

In [ ]:
features_new = ['GrLivArea', 'CRIM', 'PTRATIO', 'NOX']
X_multi_new = df[features_new]
# ... (이하 train_test_split, fit, predict, score 과정은 동일) ...
# R-squared 값이 기존보다 소폭 상승하는 것을 확인할 수 있습니다.

2.  **계수 값이 변하는 이유**
    다중 회귀에서 각 변수의 계수는 \*\*'다른 변수들이 통제(고정)되었을 때'\*\*의 순수한 영향력을 의미합니다. 단순 회귀에서의 `GrLivArea` 계수는 다른 변수(범죄율, 학생-교사 비율 등)와 `GrLivArea` 간의 숨겨진 관계까지 모두 포함된 값이었습니다. 예를 들어, 방 개수가 많은 지역은 범죄율이 낮은 경향이 있을 수 있습니다. 다중 회귀에서는 이러한 다른 변수들의 효과를 분리하여 `GrLivArea`만의 영향력을 추정하기 때문에 계수 값이 달라집니다.

### 4.3 연습 문제 정답 보기

1.  **Degree 3 모델 비교**

In [ ]:
# Degree 3 모델 생성
poly_reg_3 = make_pipeline(PolynomialFeatures(degree=3), LinearRegression())
poly_reg_3.fit(X_train, y_train)
test_rmse_3 = np.sqrt(mean_squared_error(y_test, poly_reg_3.predict(X_test)))
print(f"Degree 3 Test RMSE: {test_rmse_3:.2f}")
# Degree 2 모델보다 Test RMSE가 약간 낮아져 성능이 개선될 수 있습니다.
# 하지만 Degree 10 보다는 훨씬 좋은 성능을 보입니다.

2.  **차수별 RMSE 그래프**

In [ ]:
train_rmse_list = []
test_rmse_list = []
degree_range = range(1, 16)

for degree in degree_range:
    model = make_pipeline(PolynomialFeatures(degree=degree), LinearRegression())
    model.fit(X_train, y_train)

    train_rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
    test_rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))

    train_rmse_list.append(train_rmse)
    test_rmse_list.append(test_rmse)

# Plotly로 시각화
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(degree_range), y=train_rmse_list, mode='lines+markers', name='Train RMSE'))
fig.add_trace(go.Scatter(x=list(degree_range), y=test_rmse_list, mode='lines+markers', name='Test RMSE'))
fig.update_layout(title='차수에 따른 RMSE 변화', xaxis_title='Degree', yaxis_title='RMSE', template='plotly_white')
fig.show()
# 그래프를 보면, Train RMSE는 차수가 높아질수록 계속 감소하지만,
# Test RMSE는 특정 지점(대략 3~5)에서 최저점을 찍고 다시 증가하는 것을 볼 수 있습니다.
# 이 지점이 과적합이 시작되는 구간입니다.